## Dependencies and imports

In [1]:
#!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
#!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [2]:
#!pip install pycocotools

In [1]:
import sys
sys.path.insert(0, "../../../efficientdet-pytorch")
sys.path.insert(0, "../../../omegaconf")
sys.path.insert(0, "../")

In [2]:
import torch
import torch.nn as nn
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet

from metrics import eval_metrics

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [3]:
DATA_DIR = '/relevance2-nfs/chec/data/wheat'

In [4]:
marking = pd.read_csv(f'{DATA_DIR}/train.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

About data splitting you can read [here](https://www.kaggle.com/shonenkov/wbf-approach-for-ensemble):

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = marking[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

/relevance2-nfs/chec/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [6]:
df_folds.head()

,bbox_count,source,stratify_group,fold
image_id,,,,
00333207f,55,arvalis_1,arvalis_1_3,1
005b0d8bb,20,usask_1,usask_1_1,3
006a994f7,25,inrae_1,inrae_1_1,1
00764ad5d,41,inrae_1,inrae_1_2,0
00b5fefed,25,arvalis_3,arvalis_3_1,3


In [7]:
marking.head()

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0


In [8]:
image_files = os.listdir(os.path.join(DATA_DIR, 'train'))
all_img_ids = [x.split('.')[0] for x in image_files]
no_box_images = set(all_img_ids) - set(marking.image_id.unique())

## Albumentations

In [9]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

## Dataset

In [10]:
TRAIN_ROOT_PATH = f'{DATA_DIR}/train'

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test
        self.yxyx = True
        self.img_size = 1024

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        source =  'none'
        if self.test:
            image, boxes, source = self.load_image_and_boxes(index)
            if boxes is None:
                boxes = np.array([[0., 0., 1., 1.]])
                labels = torch.tensor([2.]).long()
            # there is only one class
            else:
                labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
            
        else:
            r = random.random()
            if True:
                image, boxes = self.load_mosaic(index)
            elif r < 0.1:
                image, boxes = self.load_image_and_boxes(index)
            elif r < 0.2:
                #print('mixup1')
                image, boxes = self.mixup1(index)
            elif r < 0.6:
                #print('mixup3')
                image, boxes = self.mixup3(index)
            else:
                #print('cutmix')
                image, boxes = self.load_cutmix_image_and_boxes(index)

            # there is only one class
            labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['source'] = source

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        if len(records) < 1:
            return image, None, 'unknown'
        source = records['source'].values[0]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        #print('>>>boxes:', boxes)
        #print('image shape:', image.shape)
        return image, boxes, source
    
    def mixup1(self, index):
        img, boxes = self.load_image_and_boxes(index)
        rindex = random.randint(0, self.image_ids.shape[0] - 1)
        rimg, rboxes = self.load_image_and_boxes(rindex)
        
        mixup_image = (img + rimg)/2
        mixup_boxes = np.concatenate([boxes, rboxes], 0)
        #print(boxes.shape, rboxes.shape, mixup_boxes.shape)
        
        return mixup_image, mixup_boxes
    
    def mixup3(self, index):
        image, boxes = self.load_image_and_boxes(index)
        rindex = random.randint(0, self.image_ids.shape[0] - 1)
        r_image, r_boxes = self.load_image_and_boxes(rindex)


        #for box in r_boxes.astype(int):
        #    cv2.rectangle(r_image,(box[0], box[1]),(box[2],  box[3]),(1, 0, 0), 3)

        imsize = image.shape[0]
        w,h = imsize, imsize
        s = imsize // 2

        xc, yc = [int(random.uniform(imsize * 0.4, imsize * 0.6)) for _ in range(2)]
        direct = random.randint(0, 3)

        result_image = image.copy()
        result_boxes = []

        if direct == 0:
            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
        elif direct == 1:  # top right
            x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
            x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
        elif direct == 2:  # bottom left
            x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
        elif direct == 3:  # bottom right
            x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)

        padw = x1a - x1b
        padh = y1a - y1b

        r_boxes[:, 0] += padw
        r_boxes[:, 1] += padh
        r_boxes[:, 2] += padw
        r_boxes[:, 3] += padh

        result_boxes.append(r_boxes)

        result_image[y1a:y2a, x1a:x2a] = (result_image[y1a:y2a, x1a:x2a] + r_image[y1b:y2b, x1b:x2b]) / 2 

        cv2.rectangle(image,(x1a, y1a),(x2a,  y2a),(0, 1, 1), 5)
        cv2.rectangle(r_image,(x1b, y1b),(x2b,  y2b),(0, 1, 1), 5)
        cv2.rectangle(result_image,(x1a, y1a),(x2a,  y2a),(0, 1, 1), 5)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]

        #for box in boxes.astype(int):
        #    cv2.rectangle(image,(box[0], box[1]),(box[2],  box[3]),(0, 0, 1), 3)
        #    cv2.rectangle(result_image,(box[0], box[1]),(box[2],  box[3]),(0, 0, 1), 3)

        #for box in result_boxes.astype(int):
        #    cv2.rectangle(result_image,(box[0], box[1]),(box[2],  box[3]),(1, 0, 0), 3)

        #ax[i][0].imshow(image)
        #ax[i][1].imshow(r_image)
        #ax[i][2].imshow(result_image)
        
        return result_image, np.concatenate([result_boxes, boxes])

    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]
        return result_image, result_boxes
    
    


In [11]:
validation_dataset = DatasetRetriever(
        image_ids=df_folds[df_folds['fold'] == 0].index.values, #df_folds[df_folds['fold'] == fold].index.values,
        marking=marking,
        transforms=get_valid_transforms(),
        test=True,
    )
val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=24,
        num_workers=24,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )

NameError: name 'collate_fn' is not defined

In [12]:
for img, target, img_id in val_loader:
    print(target[0])
    break

NameError: name 'val_loader' is not defined

In [14]:
def get_train_val_datasets(fold=0):
    val_ids_all = np.concatenate([df_folds[df_folds['fold'] == fold].index.values, np.array(list(no_box_images))], 0)
    
    train_dataset = DatasetRetriever(
        image_ids=df_folds[df_folds['fold'] != fold].index.values,
        marking=marking,
        transforms=get_train_transforms(),
        test=False,
    )

    validation_dataset = DatasetRetriever(
        image_ids=val_ids_all, #df_folds[df_folds['fold'] == fold].index.values,
        marking=marking,
        transforms=get_valid_transforms(),
        test=True,
    )
    return train_dataset, validation_dataset

In [15]:
#os.path.exists('a')

In [16]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    net.cuda()
    net = nn.DataParallel(net)
    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

In [18]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_evaluation(ifold, model_dir, by_source=False):   
    print('fold:', ifold)
    _, validation_dataset = get_train_val_datasets(ifold)

    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=24,
        num_workers=24,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )
    ckp_file = f'{model_dir}/best-fold-{ifold}.pth'
    
    if not os.path.exists(ckp_file):
        print(f'{ckp_file} not exist')
        return
    net = load_net(ckp_file)
    print(eval_metrics(net, val_loader, by_source=by_source))

In [19]:
model_dir = 'ed5-mixup'
for i in range(5):
    run_evaluation(i, model_dir, True)

fold: 0



{'all': {'best_score': 0.70729, 'best_threshold': 0.35, 0.3: 0.6988, 0.37: 0.70652, 0.4: 0.70417, 0.42: 0.70264, 0.45: 0.70008, 0.5: 0.68966, 0.55: 0.67012, 0.6: 0.64277}, 'arvalis_1': {'score': 0.61974, 'num': 212}, 'arvalis_2': {'score': 0.53475, 'num': 41}, 'arvalis_3': {'score': 0.69635, 'num': 111}, 'ethz_1': {'score': 0.71953, 'num': 149}, 'rres_1': {'score': 0.81966, 'num': 87}, 'usask_1': {'score': 0.69215, 'num': 39}, 'inrae_1': {'score': 0.80443, 'num': 36}, 'unknown': {'score': 0.95918, 'num': 49}}
fold: 1



{'all': {'best_score': 0.70229, 'best_threshold': 0.38, 0.3: 0.68718, 0.37: 0.70066, 0.4: 0.7017, 0.42: 0.7009, 0.45: 0.69951, 0.5: 0.68796, 0.55: 0.66742, 0.6: 0.6339}, 'arvalis_1': {'score': 0.61931, 'num': 211}, 'arvalis_2': {'score': 0.52611, 'num': 41}, 'arvalis_3': {'score': 0.68674, 'num': 112}, 'ethz_1': {'score': 0.70852, 'num': 149}, 'rres_1': {'score': 0.82851, 'num': 87}, 'usask_1': {'score': 0.66989, 'num': 40}, 'inrae_1': {'score': 0.82435, 'num': 35}, 'unknown': {'score': 0.93878, 'num': 49}}
fold: 2



{'all': {'best_score': 0.70269, 'best_threshold': 0.42, 0.3: 0.68708, 0.37: 0.6997, 0.4: 0.70219, 0.42: 0.70269, 0.45: 0.69993, 0.5: 0.69023, 0.55: 0.67567, 0.6: 0.64606}, 'arvalis_1': {'score': 0.61971, 'num': 211}, 'arvalis_2': {'score': 0.53549, 'num': 41}, 'arvalis_3': {'score': 0.70132, 'num': 113}, 'ethz_1': {'score': 0.69525, 'num': 148}, 'rres_1': {'score': 0.8256, 'num': 86}, 'usask_1': {'score': 0.70447, 'num': 41}, 'inrae_1': {'score': 0.82866, 'num': 35}, 'unknown': {'score': 0.91837, 'num': 49}}
fold: 3



{'all': {'best_score': 0.69332, 'best_threshold': 0.4, 0.3: 0.66859, 0.37: 0.68891, 0.4: 0.69332, 0.42: 0.69303, 0.45: 0.6909, 0.5: 0.68359, 0.55: 0.66699, 0.6: 0.63387}, 'arvalis_1': {'score': 0.6188, 'num': 211}, 'arvalis_2': {'score': 0.54678, 'num': 41}, 'arvalis_3': {'score': 0.65331, 'num': 111}, 'ethz_1': {'score': 0.7107, 'num': 150}, 'rres_1': {'score': 0.81578, 'num': 85}, 'usask_1': {'score': 0.68997, 'num': 41}, 'inrae_1': {'score': 0.78659, 'num': 35}, 'unknown': {'score': 0.89796, 'num': 49}}
fold: 4



{'all': {'best_score': 0.70225, 'best_threshold': 0.41, 0.3: 0.69286, 0.37: 0.70158, 0.4: 0.70159, 0.42: 0.7013, 0.45: 0.69805, 0.5: 0.68778, 0.55: 0.66564, 0.6: 0.62963}, 'arvalis_1': {'score': 0.61776, 'num': 210}, 'arvalis_2': {'score': 0.56159, 'num': 40}, 'arvalis_3': {'score': 0.67328, 'num': 112}, 'ethz_1': {'score': 0.71031, 'num': 151}, 'rres_1': {'score': 0.8296, 'num': 87}, 'usask_1': {'score': 0.6648, 'num': 39}, 'inrae_1': {'score': 0.82192, 'num': 35}, 'unknown': {'score': 0.93878, 'num': 49}}


In [15]:
model_dir = 'ed5-mixup'
for i in range(5):
    run_evaluation(i, model_dir)

fold: 0



{'best_score': 0.70335, 'best_threshold': 0.42, 0.3: 0.69062, 0.37: 0.70297, 0.4: 0.70219, 0.42: 0.70335, 0.45: 0.70144, 0.5: 0.69038, 0.55: 0.6688, 0.6: 0.63257}
fold: 1



{'best_score': 0.70229, 'best_threshold': 0.38, 0.3: 0.68718, 0.37: 0.70066, 0.4: 0.7017, 0.42: 0.7009, 0.45: 0.69951, 0.5: 0.68796, 0.55: 0.66742, 0.6: 0.6339}
fold: 2



{'best_score': 0.70269, 'best_threshold': 0.42, 0.3: 0.68708, 0.37: 0.6997, 0.4: 0.70219, 0.42: 0.70269, 0.45: 0.69993, 0.5: 0.69023, 0.55: 0.67567, 0.6: 0.64606}
fold: 3



{'best_score': 0.69332, 'best_threshold': 0.4, 0.3: 0.66859, 0.37: 0.68891, 0.4: 0.69332, 0.42: 0.69303, 0.45: 0.6909, 0.5: 0.68359, 0.55: 0.66699, 0.6: 0.63387}
fold: 4



{'best_score': 0.70225, 'best_threshold': 0.41, 0.3: 0.69286, 0.37: 0.70158, 0.4: 0.70159, 0.42: 0.7013, 0.45: 0.69805, 0.5: 0.68778, 0.55: 0.66564, 0.6: 0.62963}


In [22]:
model_dir = 'ed5-mixup'
for i in range(5):
    run_evaluation(i, model_dir)

fold: 0



{'best_score': 0.68881, 'best_threshold': 0.37, 0.2: 0.62152, 0.25: 0.65723, 0.3: 0.67705, 0.4: 0.68797, 0.45: 0.68421, 0.5: 0.67087, 0.55: 0.64772, 0.6: 0.60738}
fold: 1



{'best_score': 0.68512, 'best_threshold': 0.38, 0.2: 0.62734, 0.25: 0.65742, 0.3: 0.67485, 0.4: 0.68449, 0.45: 0.68066, 0.5: 0.66827, 0.55: 0.64476, 0.6: 0.6088}
fold: 2



{'best_score': 0.6853, 'best_threshold': 0.39, 0.2: 0.62664, 0.25: 0.65752, 0.3: 0.67494, 0.4: 0.68501, 0.45: 0.68102, 0.5: 0.66975, 0.55: 0.64931, 0.6: 0.61243}
fold: 3
ed5-mixup/best-fold-3.pth not exist
fold: 4
ed5-mixup/best-fold-4.pth not exist


In [23]:
model_dir = 'ed5-mixup/old'
for i in range(5):
    run_evaluation(i, model_dir)

fold: 0



{'best_score': 0.68402, 'best_threshold': 0.41, 0.2: 0.59441, 0.25: 0.63782, 0.3: 0.66302, 0.4: 0.68356, 0.45: 0.68293, 0.5: 0.67576, 0.55: 0.66103, 0.6: 0.6348}
fold: 1



{'best_score': 0.68013, 'best_threshold': 0.41, 0.2: 0.6028, 0.25: 0.63997, 0.3: 0.66319, 0.4: 0.68002, 0.45: 0.67802, 0.5: 0.67, 0.55: 0.65329, 0.6: 0.62315}
fold: 2



{'best_score': 0.68301, 'best_threshold': 0.42, 0.2: 0.60236, 0.25: 0.64114, 0.3: 0.66418, 0.4: 0.68182, 0.45: 0.6823, 0.5: 0.67698, 0.55: 0.66523, 0.6: 0.64113}
fold: 3
ed5-mixup/old/best-fold-3.pth not exist
fold: 4
ed5-mixup/old/best-fold-4.pth not exist


In [16]:
model_dir = 'effdet5-cutmix-augmix'
for i in range(5):
    run_evaluation(i, model_dir)

fold: 0



{'best_score': 0.70172, 'best_threshold': 0.41, 0.3: 0.68387, 0.37: 0.69951, 0.4: 0.70162, 0.42: 0.7013, 0.45: 0.69962, 0.5: 0.69219, 0.55: 0.67463, 0.6: 0.64799}
fold: 1



{'best_score': 0.69498, 'best_threshold': 0.43, 0.3: 0.68023, 0.37: 0.6896, 0.4: 0.69214, 0.42: 0.6936, 0.45: 0.69266, 0.5: 0.68695, 0.55: 0.67072, 0.6: 0.64138}
fold: 2



{'best_score': 0.70322, 'best_threshold': 0.41, 0.3: 0.68891, 0.37: 0.70282, 0.4: 0.70277, 0.42: 0.70293, 0.45: 0.70014, 0.5: 0.69349, 0.55: 0.67508, 0.6: 0.6431}
fold: 3



{'best_score': 0.69911, 'best_threshold': 0.4, 0.3: 0.68361, 0.37: 0.69716, 0.4: 0.69911, 0.42: 0.69838, 0.45: 0.69644, 0.5: 0.6903, 0.55: 0.67659, 0.6: 0.64595}
fold: 4



{'best_score': 0.70323, 'best_threshold': 0.39, 0.3: 0.68522, 0.37: 0.70176, 0.4: 0.70312, 0.42: 0.70202, 0.45: 0.70063, 0.5: 0.69413, 0.55: 0.67477, 0.6: 0.64396}


In [24]:
model_dir = 'effdet5-cutmix-augmix'
for i in range(5):
    run_evaluation(i, model_dir)

fold: 0



{'best_score': 0.68748, 'best_threshold': 0.41, 0.2: 0.6254, 0.25: 0.65524, 0.3: 0.67426, 0.4: 0.68737, 0.45: 0.68375, 0.5: 0.6728, 0.55: 0.65398, 0.6: 0.62392}
fold: 1



{'best_score': 0.67949, 'best_threshold': 0.38, 0.2: 0.62369, 0.25: 0.6534, 0.3: 0.67035, 0.4: 0.67868, 0.45: 0.6748, 0.5: 0.6657, 0.55: 0.6483, 0.6: 0.61535}
fold: 2



{'best_score': 0.68612, 'best_threshold': 0.41, 0.2: 0.62583, 0.25: 0.6553, 0.3: 0.67522, 0.4: 0.68564, 0.45: 0.68282, 0.5: 0.67272, 0.55: 0.65297, 0.6: 0.61867}
fold: 3



{'best_score': 0.68319, 'best_threshold': 0.38, 0.2: 0.62927, 0.25: 0.65768, 0.3: 0.67397, 0.4: 0.68317, 0.45: 0.6803, 0.5: 0.67075, 0.55: 0.65456, 0.6: 0.62169}
fold: 4



{'best_score': 0.68769, 'best_threshold': 0.38, 0.2: 0.63563, 0.25: 0.66139, 0.3: 0.67717, 0.4: 0.68747, 0.45: 0.68331, 0.5: 0.67338, 0.55: 0.65261, 0.6: 0.61956}
